In [2]:
from datetime import datetime
from collections import namedtuple
import calendar
import re
import pandas as pd

## Reading data from file

In [3]:
FILENAME = "electro_data.csv"

df = pd.read_csv(FILENAME, header=None)

In [4]:
df.head()

,0
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh


## Separating data

In [5]:
data_series = df.iloc[:, 0]

In [6]:
data_series

0           3 PM  Mon 24th-Mar-2014___0.384 kwh  
1                  5AM  15-Aug-2014___1.201  kwh 
2            __8PM Thu 20-Mar-2014____1.523 kwh  
3                 6PM  23rd-Apr-2014___0.424  kwh
4       _1AM  Friday 19th-Dec-2014___0.209  kwh  
                          ...                    
8755           _1AM  Fri 07th-Nov-2014_0.084 kwh 
8756                  _6AM 20-May-2014__1.027 kwh
8757     __2 AM  Tuesday 8th-Apr-2014___0.052 kwh
8758              9 PM 27th-Jan-2014_1.428  kwh  
8759            _12 PM 25th-Aug-2014_0.528  kwh  
Name: 0, Length: 8760, dtype: object

In [7]:
data_series = data_series.str.replace('_', ' ').str.strip()

In [8]:
data_series.head()

0       3 PM  Mon 24th-Mar-2014   0.384 kwh
1             5AM  15-Aug-2014   1.201  kwh
2          8PM Thu 20-Mar-2014    1.523 kwh
3           6PM  23rd-Apr-2014   0.424  kwh
4    1AM  Friday 19th-Dec-2014   0.209  kwh
Name: 0, dtype: object

In [29]:
TIME_PATTERN = re.compile(r'(\d+)(\s+|'')(AM|PM)')
ENERGY_PATTERN = re.compile(r'(\d{1}.\d+)(\s+|'')kwh')

Electricity = namedtuple('Electricity', ['time', 'date', 'energy'])

def extract_data(data_row):
    time = re.match(TIME_PATTERN, data_row).group()
    date = max(data_row.split(), key=len)
    energy = re.search(ENERGY_PATTERN, data_row).group()
    electricity_data = Electricity(time=time, date=date, energy=energy)
    return electricity_data

In [30]:
electro_dataset = [extract_data(data_row) for data_row in data_series]
electro_df = pd.DataFrame(electro_dataset)

In [31]:
electro_df.head()

,time,date,energy
0,3 PM,24th-Mar-2014,0.384 kwh
1,5AM,15-Aug-2014,1.201 kwh
2,8PM,20-Mar-2014,1.523 kwh
3,6PM,23rd-Apr-2014,0.424 kwh
4,1AM,19th-Dec-2014,0.209 kwh


## Cleaning energy column

In [32]:
electro_df['energy'] = electro_df['energy'].str.split('kwh').str[0].astype(float)

In [33]:
electro_df['energy'].head()

0    0.384
1    1.201
2    1.523
3    0.424
4    0.209
Name: energy, dtype: float64

## Cleaning date column

In [34]:
def format_data(data):
    if len(data) == 1:
        return f'0{data}'
    return data

In [35]:
dirty_days = electro_df['date'].str.split('-').str[0]
days_unformated = dirty_days.apply(lambda day: re.match(r'\d+', day).group())
days = [format_day(day) for day in days_unformated]

In [36]:
# def generate_short_month_names():
#     full_month_names = [calendar.month_name[i] for i in range(1, 13)]
#     short_month_names = [month[:3] for month in full_month_names]
#     return short_month_names

In [37]:
MONTH_NAMES = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

dirty_months = electro_df['date'].str.split('-').str[1]
months_unformated = dirty_months.apply(lambda month: str(MONTH_NAMES.index(month) + 1))
months = [format_data(month) for month in months_unformated]

In [38]:
years = electro_df['date'].str.split('-').str[2]

In [39]:
DATE_JOINER = '-'
dates = []

for day, month, year in zip(days, months, years):
    date_string = DATE_JOINER.join((day, month, year))
    date = datetime.strptime(date_string, '%d-%m-%Y')
    dates.append(date)

In [40]:
electro_df['date'] = dates

In [41]:
electro_df.head()

,time,date,energy
0,3 PM,2014-03-24,0.384
1,5AM,2014-08-15,1.201
2,8PM,2014-03-20,1.523
3,6PM,2014-04-23,0.424
4,1AM,2014-12-19,0.209


## Cleaning time column

In [42]:
electro_df['time']

0        3 PM
1         5AM
2         8PM
3         6PM
4         1AM
        ...  
8755      1AM
8756      6AM
8757     2 AM
8758     9 PM
8759    12 PM
Name: time, Length: 8760, dtype: object

In [43]:
PERIOD_PATTERN = re.compile(r"[a-zA-Z]+")
HOURS_PATTERN = re.compile(r"\d+")

In [44]:
periods = []
hours = []

for time in electro_df['time']:
    period = re.search(PERIOD_PATTERN, time).group(0)
    hour = re.search(HOURS_PATTERN, time).group()
    periods.append(period)
    hours.append(hour)

In [45]:
electro_df['time_hours'] = hours
electro_df['time_period'] = periods

In [46]:
electro_df.head()

,time,date,energy,time_hours,time_period
0,3 PM,2014-03-24,0.384,3,PM
1,5AM,2014-08-15,1.201,5,AM
2,8PM,2014-03-20,1.523,8,PM
3,6PM,2014-04-23,0.424,6,PM
4,1AM,2014-12-19,0.209,1,AM


In [47]:
electro_df = electro_df.drop('time', axis=1)

In [48]:
electro_df.head()

,date,energy,time_hours,time_period
0,2014-03-24,0.384,3,PM
1,2014-08-15,1.201,5,AM
2,2014-03-20,1.523,8,PM
3,2014-04-23,0.424,6,PM
4,2014-12-19,0.209,1,AM


## Getting weekday names

In [49]:
month_names = [date.strftime("%B") for date in electro_df['date']]

In [50]:
electro_df['moth'] = month_names

In [51]:
electro_df.head()

,date,energy,time_hours,time_period,moth
0,2014-03-24,0.384,3,PM,March
1,2014-08-15,1.201,5,AM,August
2,2014-03-20,1.523,8,PM,March
3,2014-04-23,0.424,6,PM,April
4,2014-12-19,0.209,1,AM,December


In [52]:
electro_df.to_csv('clean_data.csv', index=False)

In [53]:
!ls

Data cleaning challenge - vCandidate.pdf
analysis.ipynb
clean_data.csv
cleaning.ipynb
electro_data.csv
